### Time Series Analysis

Time series analysis is one of the most common types of analysis done with SQL. A `time seires` is a sequence of measurements or data points recorded in time order, often at regularly spaced intervals.

Forecasting is a common goal of time series analysis. Since time only marches forward, future values can be expressed as a function of past values, while the reverse is not ture. However, it's important to note that the past doesn't perfectly predict the future.

Still, looking at historical data can lead to insights, and developing a range of plausible outcomes is useful for planning.

#### The Retail Sales Data Set

The examples use a data set of montly US retail sales from the [Montly Retail Trade Report: Retail and Food Services Sales: Excel (1992-2020)](https://www.census.gov/retail/index.html#mrts).

The data in this report is uesd as an economic indicator to understand trends in US consumer spending patterns.

While gross domestic product(GDP) figures are published quarterly, this retail sales data is published monthly, so it is also used to help predict GDP.

The data spans from 1992 to 2020 and includes both total sales as well as details for subcategories of retail sales.

Sales figures are in millions of US dollars.

In [11]:
import pandas as pd
import sqlite3

df = pd.read_csv('us_retail_sales.csv')

conn = sqlite3.connect("sql-analysis.db")
df.to_sql('retail_sales', conn)

In [13]:
import matplotlib as mpl
import matplotlib.pyplot as plt

- Simple Trends

In [37]:
query_01 = """
        SELECT sales_month, sales
        FROM retail_sales
        WHERE kind_of_business = 'Retail and food services sales, total'
        """

sql_01 = pd.read_sql(query_01, conn)
sql_01.head()

,sales_month,sales
0,1992-01-01,146376.0
1,1992-02-01,147079.0
2,1992-03-01,159336.0
3,1992-04-01,163669.0
4,1992-05-01,170068.0


<img align="left" width="437" alt="Screen Shot 2022-04-18 at 4 28 01 PM" src="https://user-images.githubusercontent.com/73784742/163780536-143cd751-373d-4d84-9e19-763575601ccf.png">

In [ ]:
'''
plt.subplots(figsize=(12, 8))
plt.plot(sql_01['sales_month'], sql_01['sales'])

plt.title("Monthly Retail and Food Services Sales")
plt.xlabel("Month")
plt.ylabel("$MM Sales")

plt.xticks(ticks=sql_01['sales_month'], labels=sql_01['sales_month'], rotation=45)
plt.locator_params(axis='x', nbins=len(sql_01['sales_month'])/12)

plt.show();
'''

In [38]:
query_02 = """
        SELECT strftime('%Y', sales_month) as sales_year, SUM(sales) as sales
        FROM retail_sales
        WHERE kind_of_business = 'Retail and food services sales, total'
        GROUP BY sales_year
        """

sql_02 = pd.read_sql(query_02, conn)
sql_02.head()

,sales_year,sales
0,1992,2014102.0
1,1993,2153095.0
2,1994,2330235.0
3,1995,2450628.0
4,1996,2603794.0


<img align="left" width="435" alt="Screen Shot 2022-04-18 at 4 31 29 PM" src="https://user-images.githubusercontent.com/73784742/163781000-f42f328b-51a9-4131-8819-1eb89433d8b5.png">

In [ ]:
'''
plt.subplots(figsize=(12, 8))
plt.plot(sql_02['sales_year'], sql_02['sales'])

plt.gcf().axes[0].yaxis.get_major_formatter().set_scientific(False)

plt.title("Yearly Total Retail and Food Services Sales")
plt.xlabel("Year")
plt.ylabel("$MM Sales")
plt.xticks(ticks=sql_02['sales_year'], labels=sql_02['sales_year'], rotation=45)
plt.locator_params(axis='x', nbins=len(sql_02['sales_year']))

plt.show();
'''

- Comparing Components

In [65]:
query_03 = """
        SELECT strftime('%Y', sales_month) as sales_year, kind_of_business, SUM(sales) as sales
        FROM retail_sales
        WHERE kind_of_business IN ('Book stores', 'Sporting goods stores', 'Hobby, toy, and game stores')
        GROUP BY 1, 2
        """

sql_03 = pd.read_sql(query_03, conn)
sql_03

,sales_year,kind_of_business,sales
0,1992,Book stores,8327.0
1,1992,"Hobby, toy, and game stores",11251.0
2,1992,Sporting goods stores,15583.0
3,1993,Book stores,9108.0
4,1993,"Hobby, toy, and game stores",11651.0
...,...,...,...
82,2019,"Hobby, toy, and game stores",16261.0
83,2019,Sporting goods stores,43808.0
84,2020,Book stores,6425.0
85,2020,"Hobby, toy, and game stores",17287.0


<img align="left" width="435" alt="Screen Shot 2022-04-18 at 4 40 33 PM" src="https://user-images.githubusercontent.com/73784742/163782175-0fc4e8a8-54a5-4356-8248-d27c7c5fed88.png"> 